In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("."))

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import TensorDataset, DataLoader, ConcatDataset

import utils
import environments

%load_ext autoreload
%autoreload 2

In [ ]:
config_CLogisticW = {
    'env_name': '2d_classification',
    'npz_file': 'sine_dataset_3.npz',
    'root': '../datasets/synthetic/',
    'train_ratio': 0.8,
}
config_PermW = {
    'permutation_size': 32,
    'number_tasks': 5,
    'env_name': 'permuted',
    'dataset_to_permute': 'cifar10',
    'data_size': 32,
    'num_classes': 10,
    'root': '../datasets',
}  
config_SplitW = {
    'split_type': 'classes',
    'number_tasks': 5,
    'env_name': 'split',
    'dataset_to_split': 'mnist',
    'data_size': 28,
    'num_classes': 10,
    'root': '../datasets',
}
config_ShuffleW = {
    'shuffle_fraction': 0.3,
    'number_tasks': 5,
    'env_name': 'shuffled',
    'dataset_to_shuffle': 'cifar10',
    'data_size': 32,
    'num_classes': 10,
    'root': '../datasets',
}
config_clear = {
    'env_name': 'clear',
    'root': '../../continual-rl/datasets/clear100',
    'download': False,
} # careful, need to download the dataset manually
config_MD5 = {
    'env_name': 'multidataset',
    'root': '../../continual-rl/datasets/',
}


In [26]:
env_args = config_clear.copy()
env_args_obj = utils.Dict2Args(**env_args)

env, env_name = environments.get_environment_from_name(
    env_args['env_name'], env_args_obj
)

Files already downloaded and verified
Files already downloaded and verified


In [24]:
!ls

README.md    credentials.py	  networks.py		   utils.py
__init__.py  environments.py	  requirements.txt	   viz_utils.py
__pycache__  logistic_data.ipynb  test_environments.ipynb


In [9]:
print(f"Training samples: {len(env.init_single_task(0, train=True))}")
print(f"Test samples: {len(env.init_single_task(0, train=False))}")

Training samples: 12665
Test samples: 2115


In [10]:
# get a batch of data and plot it
data_batch = env.sample_batch_joint(10, train=True)
data_batch

[tensor([[[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           ...,
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242]]],
 
 
         [[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           ...,
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242]]],
 
 
         [[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
       